In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/GRAPH/TeamProject/KDD/whoiswho-top-solutions/incorrect_assignment_detection/GCN

/content/drive/MyDrive/GRAPH/TeamProject/KDD/whoiswho-top-solutions/incorrect_assignment_detection/GCN


In [ ]:
!pip install -r requirements.txt

In [ ]:
!python build_graph.py

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
done clean pubs
done loading embeddings
finish
done process dataset/train_author.json


In [ ]:
!python train.py

[2024-05-10 07:48:45] {train.py:83} INFO - Namespace(train_dir='dataset/train.pkl', eval_dir=None, test_dir=None, saved_dir='saved_model', log_name='log', epochs=50, seed=1, lr=0.0005, min_lr=0.0002, bs=1, input_dim=768, output_dim=768, verbose=1, usecoo=False, usecov=False, threshold=0)
[2024-05-10 07:48:49] {train.py:100} INFO - # Batch: 545 - 545.0
[2024-05-10 07:48:49] {train.py:105} INFO - max_step: 5450, 545, 1, 50
100% 545/545 [00:11<00:00, 46.79it/s]
[2024-05-10 07:49:00] {train.py:152} INFO - Epoch:0 Overall loss: 0.171591 
100% 234/234 [00:00<00:00, 280.69it/s]
[2024-05-10 07:49:01] {train.py:195} INFO - Epoch: 0 Auc: 0.557309 Maps: 0.685582 Max-Auc: -1.000000 Max-Maps: -1.000000
[2024-05-10 07:49:02] {train.py:205} INFO - ***************** Epoch: 0 Max Auc: 0.557309 Maps: 0.685582 *******************
100% 545/545 [00:08<00:00, 64.58it/s]
[2024-05-10 07:49:10] {train.py:152} INFO - Epoch:1 Overall loss: 0.169908 
100% 234/234 [00:00<00:00, 391.28it/s]
[2024-05-10 07:49:11] {t

In [ ]:
# Load the validation data
with open('./dataset/valid.pkl', 'rb') as f:
    valid_data_tuple = pickle.load(f)


# Inspect the structure of valid_data_tuple
print(f"Type of valid_data_tuple: {type(valid_data_tuple)}")
print(f"Length of valid_data_tuple: {len(valid_data_tuple)}")

# Inspect each element
for i, element in enumerate(valid_data_tuple):
    print(f"Element {i} type: {type(element)}")
    if isinstance(element, (list, tuple)):
        print(f"Element {i} length: {len(element)}")
        if len(element) > 0:
            print(f"First item in element {i}: {element[0]}")
        else:
            print(f"Element {i} is empty")
    else:
        print(f"Element {i}: {element}")

# Specifically inspect the first element which we expect to be DataBatch
data_batch = valid_data_tuple[0]
print(f"data_batch type: {type(data_batch)}")
print(f"data_batch attributes: {data_batch.__dict__.keys()}")

Type of valid_data_tuple: <class 'list'>
Length of valid_data_tuple: 370
Element 0 type: <class 'tuple'>
Element 0 length: 4
First item in element 0: DataBatch(x=[382, 768], edge_index=[2, 64604], edge_attr=[64604, 3], batch=[382])
Element 1 type: <class 'tuple'>
Element 1 length: 4
First item in element 1: DataBatch(x=[191, 768], edge_index=[2, 27444], edge_attr=[27444, 3], batch=[191])
Element 2 type: <class 'tuple'>
Element 2 length: 4
First item in element 2: DataBatch(x=[43, 768], edge_index=[2, 764], edge_attr=[764, 3], batch=[43])
Element 3 type: <class 'tuple'>
Element 3 length: 4
First item in element 3: DataBatch(x=[29, 768], edge_index=[2, 322], edge_attr=[322, 3], batch=[29])
Element 4 type: <class 'tuple'>
Element 4 length: 4
First item in element 4: DataBatch(x=[109, 768], edge_index=[2, 3392], edge_attr=[3392, 3], batch=[109])
Element 5 type: <class 'tuple'>
Element 5 length: 4
First item in element 5: DataBatch(x=[37, 768], edge_index=[2, 542], edge_attr=[542, 3], batch

AttributeError: 'tuple' object has no attribute '__dict__'

In [ ]:
# Extract the Batch and node IDs
data_batch = valid_data_tuple[0][0]  # First element of the first tuple
ids = valid_data_tuple[0][2]  # Second element of the first tuple
node_ids = valid_data_tuple[0][3]  # Fourth element of the first tuple

# Extract features and labels from the DataBatch
num_features = data_batch.x.shape[1]
num_classes = 2  # As we are predicting if the connection is correct or not

In [ ]:
ids

'efQ8FQ1i'

In [ ]:
# valid.pkl를 이용해서 학습한 모델을 평가하고 그 결과를 ind_valid_author_sumit.json 형태로 저장
# evaluate를 시행하는 코드
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCNModel(nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.fc(x)

        return F.sigmoid(x)


In [ ]:
import pickle
import torch
import json
from torch_geometric.data import Data, Batch

# Load validation data
with open('./dataset/valid.pkl', 'rb') as f:
    valid_data_tuple = pickle.load(f)

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the entire model object
model = torch.load('./saved_model/model_29.pt', map_location=device)
model.eval()

# Prepare a dictionary to hold the results
results = {}

# Iterate over each batch in the validation data
for batch_idx, data_batch in enumerate(valid_data_tuple):
    data, target_nodes, ids, batch_node_ids = data_batch[0], data_batch[1], data_batch[2], data_batch[3]  # Assuming node IDs are in the fourth position of the tuple

    # Move data to the same device as the model
    data = data.to(device)

    with torch.no_grad():
        output = model(data.x, data.edge_index)
        predictions = output.squeeze().tolist()  # Convert predictions to a list of values

        # Use a batch identifier key such as "efQ8FQ1i"
        batch_key = ids
        results[batch_key] = {}

        for idx, node_id in enumerate(batch_node_ids):
            results[batch_key][node_id] = predictions[idx]  # Map each prediction to the corresponding node ID

# Save the results to ind_valid_author_submit.json
with open('./dataset/ind_valid_author_submit.json', 'w') as f:
    json.dump(results, f, indent=4)

print("Evaluation results saved to ind_valid_author_submit.json")

Evaluation results saved to ind_valid_author_submit.json
